In [1]:
%pip install -q -U transformers datasets accelerate peft trl bitsandbytes
%pip install einops

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install flash-attn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import re
import torch
from spacy.lang.en import English
from datasets import (load_dataset, Dataset)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
import json
import random
import numpy as np

random.seed(1)


/opt/conda/envs/pii_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Set the enviornment as Hugging Face Token
os.environ["HF_TOKEN"] = ""

# Select Models
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "pii-llama3-8b-4bit"
final_model_name = new_model + "-merged"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token=tokenizer.eos_token
test_data_percentage = 0.15


nlp = English()
english_tokenizer = nlp.tokenizer

pii_labels = ['NAME_STUDENT', 'EMAIL', 'ID_NUM', 'USERNAME', 'URL_PERSONAL', 'STREET_ADDRESS', 'PHONE_NUM']
pii_labels_pattern = '|'.join(pii_labels)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def extract_personal_formation(text):
    names, emails, usernames, id_numbers, phone_numbers, urls, street_addresses = [], [], [], [], [], [], []
    outputs = re.split(r',?\n', text)
    outputs = [re.sub(r'((^\d+\.)|(\*))\s*', '', output) for output in outputs]
    for output in outputs:
        result = re.search(f"[^)(\s]+\s?\(({pii_labels_pattern})\)", output)
        if not result: continue

        data = output.replace(f"({result.group(1)})", "").strip()
        label = result.group(1)
        if label == "NAME_STUDENT":
            names.append(data)
        elif label == "EMAIL":
            emails.append(data)
        elif label == "USERNAME":
            usernames.append(data)
        elif label == "ID_NUM":
            id_numbers.append(data)
        elif label == "PHONE_NUM":
            phone_numbers.append(data)
        elif label == "URL_PERSONAL":
            urls.append(data)
        elif label == "STREET_ADDRESS":
            street_addresses.append(data)
    
    return (names, emails, usernames, id_numbers, phone_numbers, urls, street_addresses)

def format_pii_output(text):
    if not text: return ""
    
    names, emails, usernames, id_numbers, phone_numbers, urls, street_addresses = extract_personal_formation(text)
    names = "\n".join([f"* {name}" for name in names]) if names else "* None"
    emails = "\n".join([f"* {email}" for email in emails]) if emails else "* None"
    usernames = "\n".join([f"* {username}" for username in usernames]) if usernames else "* None"
    id_numbers = "\n".join([f"* {id_number}" for id_number in id_numbers]) if id_numbers else "* None"
    phone_numbers = "\n".join([f"* {phone_number}" for phone_number in phone_numbers]) if phone_numbers else "* None"
    urls = "\n".join([f"* {url}" for url in urls]) if urls else "* None"
    street_addresses = "\n".join([f"* {street_address}" for street_address in street_addresses]) if street_addresses else "* None"

    return f'''List of Names:
{names}

List of Emails:
{emails}

List of Usernames:
{usernames}

List of ID Numbers:
{id_numbers}

List of Phone Numbers:
{phone_numbers}

List of URLs:
{urls}

List of Street Addresses:
{street_addresses}'''

def format_prompt(text: str, answer: str = ''):
  if answer != '':
    answer = format_pii_output(answer)
    answer += "\n" + tokenizer.eos_token

  return f'''<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to identify and list any personal information in the input text. Please, list the following personal information.

List of Names:
* first full or partial name of a person

List of Emails:
* email address of a person

List of Usernames:
* username of a person

List of ID Numbers:
* number or sequence of characters that could be used to identify a person like the student ID or a social security number

List of Phone Numbers:
* phone number associated with a person

List of URLs:
* URL that might be used to identify a person

List of Street Addresses:
* full or partial street address that is associated with the person, such as a home address

### Input:
Text: {text}

### Response:
{answer}'''

## Loading Alpaca

In [4]:
alpaca_prompt = '''<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}'''

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [5]:
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
alapca_dataset = dataset.map(formatting_prompts_func, batched = True,)
alapca_dataset = pd.DataFrame(alapca_dataset)
alapca_dataset

,output,input,instruction,text
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.,<|begin_of_text|>Below is an instruction that ...
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?,<|begin_of_text|>Below is an instruction that ...
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.,<|begin_of_text|>Below is an instruction that ...
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?,<|begin_of_text|>Below is an instruction that ...
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...,<|begin_of_text|>Below is an instruction that ...
...,...,...,...,...
51755,Yes,Text: John went out for a walk with his dog Ro...,You will be given a piece of text about an eve...,<|begin_of_text|>Below is an instruction that ...
51756,True,Text: Michael Jordan is an American former pro...,You will be given a paragraph of text with var...,<|begin_of_text|>Below is an instruction that ...
51757,True,Text: A tree fell over in the wind and caused ...,You will be given a piece of text about an eve...,<|begin_of_text|>Below is an instruction that ...
51758,Backwards,"Steps: ['She takes out her books', 'The teache...",I will give you a list of steps. You need to ...,<|begin_of_text|>Below is an instruction that ...


## Load Custom PII Dataset

In [6]:
def extract_training_data_from_documents(documents, window_size=10000):
    temp_label_buffer = ''
    labels = ['NAME_STUDENT', 'EMAIL', 'ID_NUM', 'USERNAME', 'URL_PERSONAL', 'STREET_ADDRESS', 'PHONE_NUM']
    data_finetune_full = []
    data_finetune_no_answer = []

    for document in documents:
        tokens = document['tokens']
        labels_per_token = document['labels']

        token_chunks = [tokens[i:i + window_size] for i in range(0, len(tokens), window_size)]
        label_chunks = [labels_per_token[i:i + window_size] for i in range(0, len(labels_per_token), window_size)]

        for chunk_index in range(len(token_chunks)):
            text = ''
            output = ''
            answers = []

            for token_index in range(len(token_chunks[chunk_index])):
                token = token_chunks[chunk_index][token_index]
                label = label_chunks[chunk_index][token_index]

                if token in ['.', ',', '!', '?', "'", '(', ')', ' ']:
                    text += token
                elif '#' in token:
                    new_token = token.replace('#','')
                    text += new_token
                else:
                    text += ' ' + token

                for label_type in labels:
                    if label == 'B-' + label_type or label == 'I-' + label_type:
                        if len(temp_label_buffer) > 1:
                            temp_label_buffer += (' ' + token)
                        else:
                            temp_label_buffer += token
                    elif len(temp_label_buffer) > 0 and label == 'O' and (label_chunks[chunk_index][token_index - 1] == 'B-' + label_type or
                                            label_chunks[chunk_index][token_index - 1] == 'I-' + label_type):
                        temp_label_buffer += (' ' + f'({label_type})')
                        if temp_label_buffer not in answers:
                            answers.append(temp_label_buffer)

                if label == 'O':
                    temp_label_buffer = ''

            for answer in answers:
                output += answer + '\n'

            data_finetune_full.append({'text': format_prompt(text, output), 'output': output, 'full_text': text})
            data_finetune_no_answer.append({'text': format_prompt(text, ''), 'output': output, 'full_text': text})

    return data_finetune_full, data_finetune_no_answer

def retokenize(mod_30k_data):
  nlp = English()
  # Create a Tokenizer with the default settings for English
  # including punctuation rules and exceptions
  tokenizer = nlp.tokenizer

  for k in range(len(mod_30k_data)):
    new_tokens = []
    tokens = tokenizer(mod_30k_data[k]['source_text'])
    labels = ['O'] * len(tokens)

    for i in range(len(labels)):
      for label in mod_30k_data[k]['privacy_mask']:
        if str(tokens[i]) in label['value']:
          labels[i] = 'I-'+label['label']
      new_tokens.append(str(tokens[i]))

    mod_30k_data[k]['labels'] = labels
    mod_30k_data[k]['tokens'] = new_tokens

  return mod_30k_data

def replace_strings(lst, old_string, new_string):
    return [new_string if item == old_string else item for item in lst]

def label_replace(mod_30k_data):
  old_label = ['LASTNAME1','LASTNAME2','SOCIALNUMBER','TEL','DRIVERLICENSE','STREET','BUILDING','PASSPORT','GIVENNAME1','GIVENNAME2','LASTNAME3','STATE','POSTCODE','CITY','IDCARD','IP']
  new_label = ['NAME_STUDENT','NAME_STUDENT','ID_NUM','PHONE_NUM','ID_NUM','STREET_ADDRESS','STREET_ADDRESS','ID_NUM','NAME_STUDENT','NAME_STUDENT','NAME_STUDENT','STREET_ADDRESS','STREET_ADDRESS','STREET_ADDRESS','ID_NUM','URL_PERSONAL']

  address_builder = ''

  for i in range(len(mod_30k_data)):
    for k in range(len(mod_30k_data[i]['privacy_mask'])):
      for t in range(len(new_label)):
        if mod_30k_data[i]['privacy_mask'][k]['label'] == old_label[t]:
          if mod_30k_data[i]['privacy_mask'][k]['label'] == 'IP' or mod_30k_data[i]['privacy_mask'][k]['label']=='URL_PERSONAL':
            mod_30k_data[i]['privacy_mask'][k]['value'] = get_new_url()

          mod_30k_data[i]['privacy_mask'][k]['label'] = new_label[t]
          continue

  return mod_30k_data

def bert_300_gen(documents):
  labels = ['NAME_STUDENT', 'EMAIL', 'ID_NUM', 'USERNAME', 'URL_PERSONAL', 'STREET_ADDRESS', 'PHONE_NUM']
  temp_output_arr = []
  temp_output = ''


  for t in range(len(documents)):
    for i in range(len(documents[t]['privacy_mask'])):
      if documents[t]['privacy_mask'][i]['label'] in labels:
        if documents[t]['privacy_mask'][i]['value']+' ('+documents[t]['privacy_mask'][i]['label']+')\n' not in temp_output_arr:
          temp_output_arr.append(documents[t]['privacy_mask'][i]['value']+' ('+documents[t]['privacy_mask'][i]['label']+')\n')

    for strings in temp_output_arr:
      temp_output+=strings

    documents[t]['output'] = temp_output
    documents[t]['text'] = format_prompt(documents[t]['source_text'],temp_output)
    documents[t]['testing_text'] = format_prompt(documents[t]['source_text'],'')
    documents[t]['full_text'] = documents[t]['source_text']


    temp_output_arr = []
    temp_output = ''

  return documents

In [7]:
import pandas as pd
data_path = "30k_english_instruction.json"
jsonObj = pd.read_json(path_or_buf=data_path, lines=True)

mod_30k_data = jsonObj.to_dict('records')
mod_30k_data = mod_30k_data[0]
mod_30k_data = label_replace(mod_30k_data)
data = bert_300_gen(mod_30k_data)

In [8]:
mod_30k_data = list(mod_30k_data.values())
english_pii_dataset = pd.DataFrame(mod_30k_data)
english_pii_dataset

,source_text,target_text,privacy_mask,span_labels,mbert_text_tokens,mbert_bio_labels,id,language,set,output,final_text,testing_text,text,full_text
0,Subject: Group Messaging for Admissions Proces...,Subject: Group Messaging for Admissions Proces...,"[{'value': 'wynqvrh053', 'start': 287, 'end': ...","[[440, 453, ""USERNAME""], [430, 437, ""TIME""], [...","[Sub, ##ject, :, Group, Mess, ##aging, for, Ad...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40767A,English,train,wynqvrh053 (USERNAME)\nluka.burg (USERNAME)\nq...,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,Subject: Group Messaging for Admissions Proces...
1,- Meeting at 2:33 PM\n- N23 - Meeting at 11:29...,- Meeting at [TIME]\n- [USERNAME] - Meeting at...,"[{'value': '2:33 PM', 'start': 13, 'end': 20, ...","[[74, 81, ""TIME""], [50, 60, ""USERNAME""], [40, ...","[-, Meeting, at, 2, :, 33, PM, -, N, ##23, -, ...","[O, O, O, B-TIME, I-TIME, I-TIME, I-TIME, O, O...",40767B,English,train,N23 (USERNAME)\nwennmann27 (USERNAME)\n,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,- Meeting at 2:33 PM\n- N23 - Meeting at 11:29...
2,Subject: Admission Notification - Great Britai...,Subject: Admission Notification - Great Britai...,"[{'value': '5:24am', 'start': 263, 'end': 269,...","[[395, 407, ""SOCIALNUMBER""], [358, 375, ""EMAIL...","[Sub, ##ject, :, Ad, ##mission, Not, ##ificati...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",40768A,English,train,Balloi (NAME_STUDENT)\nEckrich (NAME_STUDENT)\...,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,Subject: Admission Notification - Great Britai...
3,Card: KB90324ER\n Country: GB\n Building: ...,Card: [IDCARD]\n Country: [COUNTRY]\n Buil...,"[{'value': 'KB90324ER', 'start': 6, 'end': 15,...","[[390, 393, ""STATE""], [368, 378, ""CITY""], [346...","[Card, :, KB, ##90, ##32, ##4, ##ER, \, n, Cou...","[O, O, B-IDCARD, I-IDCARD, I-IDCARD, I-IDCARD,...",40768B,English,train,KB90324ER (ID_NUM)\n163 (STREET_ADDRESS)\nCony...,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,Card: KB90324ER\n Country: GB\n Building: ...
4,"N, WA14 5RW\n Password: r]iD1#8\n\n...and so...","N, WA14 5RW\n Password: [PASS]\n\n...and so ...","[{'value': 'r]iD1#8', 'start': 26, 'end': 33, ...","[[336, 352, ""DATE""], [26, 33, ""PASS""]]","[N, ,, W, ##A, ##14, 5, ##R, ##W, \, n, Pass, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PASS...",40768C,English,train,,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,"N, WA14 5RW\n Password: r]iD1#8\n\n...and so..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29903,e of Birth: [18/01/1962]\n - Passport Number...,e of Birth: [[BOD]]\n - Passport Number: [[P...,"[{'value': '18/01/1962', 'start': 13, 'end': 2...","[[379, 384, ""POSTCODE""], [360, 361, ""TIME""], [...","[e, of, Birth, :, [, 18, /, 01, /, 1962, ], -,...","[O, O, O, O, O, B-BOD, I-BOD, I-BOD, I-BOD, I-...",53616B,English,train,MI (ID_NUM)\n031412682 (ID_NUM)\n350804398 (ID...,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is an instruction that ...,e of Birth: [18/01/1962]\n - Passport Number...
29904,"2022, in New York City. The assessment include...","2022, in New York City. The assessment include...",[],[],"[2022, ,, in, New, York, City, ., The, assessm...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",53616C,English,train,,<s>[INST] <<SYS>>\nYou are an intelligent assi...,<|begin_of_text|>Below is an instruction that ...,<|begin_of_text|>Below is 

## Load PII Dataset

In [9]:
def extract_training_data_from_documents(documents, window_size=10000):
    temp_label_buffer = ''
    labels = ['NAME_STUDENT', 'EMAIL', 'ID_NUM', 'USERNAME', 'URL_PERSONAL', 'STREET_ADDRESS', 'PHONE_NUM']
    data_finetune_full = []
    data_finetune_no_answer = []

    for document in documents:
        tokens = document['tokens']
        labels_per_token = document['labels']

        token_chunks = [tokens[i:i + window_size] for i in range(0, len(tokens), window_size)]
        label_chunks = [labels_per_token[i:i + window_size] for i in range(0, len(labels_per_token), window_size)]

        for chunk_index in range(len(token_chunks)):
            text = ''
            output = ''
            answers = []

            for token_index in range(len(token_chunks[chunk_index])):
                token = token_chunks[chunk_index][token_index]
                label = label_chunks[chunk_index][token_index]

                if token in ['.', ',', '!', '?', "'", '(', ')', ' ']:
                    text += token
                else:
                    text += ' ' + token

                for label_type in labels:
                    if label == 'B-' + label_type or label == 'I-' + label_type:
                        if len(temp_label_buffer) > 1:
                            temp_label_buffer += (' ' + token)
                        else:
                            temp_label_buffer += token
                    elif label == 'O' and (label_chunks[chunk_index][token_index - 1] == 'B-' + label_type or
                                            label_chunks[chunk_index][token_index - 1] == 'I-' + label_type):
                        temp_label_buffer += (' ' + f'({label_type})')
                        if temp_label_buffer not in answers:
                            answers.append(temp_label_buffer)

                if label == 'O':
                    temp_label_buffer = ''

            for answer in answers:
                output += answer + '\n'

            output = output.strip()
            data_finetune_full.append({'text': format_prompt(text, output), 'output': output, 'full_text': text})
            data_finetune_no_answer.append({'text': format_prompt(text, ''), 'output': output, 'full_text': text})

    return data_finetune_full, data_finetune_no_answer

In [10]:
train_data_path = "train.json"
test_data_path = "test.json"
num_test_data = 13

# Loading Dataset
with open(train_data_path) as file:
    train_data_json = json.load(file)

with open(test_data_path ) as file:
    test_data_json = json.load(file)

print("Training Data: ", len(train_data_json))
print("Test Data: ", len(test_data_json))

# TODO: Only using the first samples as test data, which are only for predicting names. Need to include the other types of information in our data as well, and also add data from the other PII dataset into the testing data
training_data, testing_data = extract_training_data_from_documents(train_data_json, 1000)

# Convert the list of dictionaries into a DataFrame
essay_pii_dataset = pd.DataFrame(training_data)
essay_pii_dataset

Training Data:  6807
Test Data:  10


,text,output,full_text
0,<|begin_of_text|>Below is an instruction that ...,Nathalie Sylla (NAME_STUDENT),Design Thinking for innovation reflexion - Av...
1,<|begin_of_text|>Below is an instruction that ...,Diego Estrada (NAME_STUDENT),Diego Estrada \n\n Design Thinking Assignment...
2,<|begin_of_text|>Below is an instruction that ...,Gilberto Gamboa (NAME_STUDENT),Reporting process \n\n by Gilberto Gamboa \n\...
3,<|begin_of_text|>Below is an instruction that ...,Sindy Samaca (NAME_STUDENT),Design Thinking for Innovation \n\n Sindy Sam...
4,<|begin_of_text|>Below is an instruction that ...,,", I \n\n would like to use the learning launch..."
...,...,...,...
7952,<|begin_of_text|>Below is an instruction that ...,,EXAMPLE – JOURNEY MAP \n\n THE CHALLENGE ...
7953,<|begin_of_text|>Below is an instruction that ...,,Why Mind Mapping? \n\n Mind maps are graphica...
7954,<|begin_of_text|>Below is an instruction that ...,,"Challenge \n\n So, a few months back, I had c..."
7955,<|begin_of_text|>Below is an instruction that ...,,Brainstorming \n\n Challenge & Selection \n\n...


## Label Distribution

In [11]:
# Counting number of occurances of each label
pii_labels = ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']
num_labels = {label: 0 for label in pii_labels}

for label in pii_labels:
  for output in essay_pii_dataset['output']:
    num_labels[label] += output.count(label)

print("Essay Dataset PII Occurrances:\n")
num_labels

Essay Dataset PII Occurrances:



{'NAME_STUDENT': 988,
 'EMAIL': 26,
 'USERNAME': 5,
 'ID_NUM': 67,
 'PHONE_NUM': 4,
 'URL_PERSONAL': 97,
 'STREET_ADDRESS': 2}

In [12]:
# Counting number of occurances of each label
pii_labels = ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']
num_labels = {label: 0 for label in pii_labels}

for label in pii_labels:
  for output in english_pii_dataset['output']:
    num_labels[label] += output.count(label)

print("English Instruction Dataset PII Occurrances:\n")
num_labels

English Instruction Dataset PII Occurrances:



{'NAME_STUDENT': 21559,
 'EMAIL': 9716,
 'USERNAME': 10867,
 'ID_NUM': 36661,
 'PHONE_NUM': 7262,
 'URL_PERSONAL': 8145,
 'STREET_ADDRESS': 36159}

## Merge Datasets

In [32]:
# Getting portions from PII Essay Dataset in the start, middle, and end of the dataset and combining them with the English Instruction Dataset
pii_dataset = pd.concat([
    essay_pii_dataset[-1000:-1],
    essay_pii_dataset[0:2000],
    essay_pii_dataset[int(len(essay_pii_dataset) * 0.5):int(len(essay_pii_dataset) * 0.5) + 2000],
    english_pii_dataset
])

In [74]:
# Splitting PII datasets into training and testing datasets
test_indexes = set(random.sample(range(len(pii_dataset)), int(len(pii_dataset) * test_data_percentage)))

pii_train_dataset = pii_dataset[~pii_dataset.index.isin(test_indexes)]
pii_test_dataset = pii_dataset[pii_dataset.index.isin(test_indexes)]

In [75]:
# Counting number of occurances of each label
pii_labels = ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']
num_labels = {label: 0 for label in pii_labels}

for label in pii_labels:
  for output in pii_train_dataset['output']:
    num_labels[label] += output.count(label)

print("Train Dataset PII Occurrances:\n")
num_labels

Train Dataset PII Occurrances:



{'NAME_STUDENT': 18850,
 'EMAIL': 8230,
 'USERNAME': 9220,
 'ID_NUM': 31119,
 'PHONE_NUM': 6135,
 'URL_PERSONAL': 7061,
 'STREET_ADDRESS': 30729}

In [76]:
# Counting number of occurances of each label
pii_labels = ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']
num_labels = {label: 0 for label in pii_labels}

for label in pii_labels:
  for output in pii_test_dataset['output']:
    num_labels[label] += output.count(label)

print("Test Dataset PII Occurrances:\n")
num_labels

Test Dataset PII Occurrances:



{'NAME_STUDENT': 3484,
 'EMAIL': 1507,
 'USERNAME': 1652,
 'ID_NUM': 5598,
 'PHONE_NUM': 1131,
 'URL_PERSONAL': 1157,
 'STREET_ADDRESS': 5431}

In [77]:
train_dataset = pd.concat([alapca_dataset, pii_train_dataset])
train_dataset = Dataset.from_pandas(train_dataset)
train_dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text', 'full_text', 'source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'final_text', 'testing_text', '__index_level_0__'],
    num_rows: 81459
})

In [37]:
print(train_dataset['text'][70000])

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to identify and list any personal information in the input text. Please, list the following personal information.

List of Names:
* first full or partial name of a person

List of Emails:
* email address of a person

List of Usernames:
* username of a person

List of ID Numbers:
* number or sequence of characters that could be used to identify a person like the student ID or a social security number

List of Phone Numbers:
* phone number associated with a person

List of URLs:
* URL that might be used to identify a person

List of Street Addresses:
* full or partial street address that is associated with the person, such as a home address

### Input:
Text: {
  "transcript_records": {
    "student_name": "949140063",
    "courses": [
      {
        "course_name": "Mathematics",
     

## Model Training

In [42]:
#To reduce the VRAM usage we will load the model in 4 bit precision and we will do quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    #Quant type
    #We will use the "nf4" format this was introduced in the QLoRA paper
    bnb_4bit_quant_type="nf4",
    #As the model weights are stored using 4 bits and when we want to compute its only going to use 16 bits so we have more accuracy
    bnb_4bit_compute_dtype=torch.float16,
    #Quantization parameters are quantized
    bnb_4bit_use_double_quant=False,
)

peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=256,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"
                , "down_proj", "lm_head"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0},
    use_auth_token=True,
    use_flash_attention_2=True, # use flash attention 2
)


model.config.use_cache = True

# Cast the layernorm in fp32, make output embedding layer require grads, add the upcasting of the lmhead to fp32
#prepare_model_for_kbit_training---> This function basically helps to built the best model possible
model = prepare_model_for_kbit_training(model,use_gradient_checkpointing=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/envs/pii_env/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


In [43]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
16.824 GB of memory reserved.


In [44]:
# Set training arguments
training_arguments = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
)


# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field = "text",
    peft_config=peft_config,
    dataset_num_proc = 2,
    max_seq_length=2048,# In dataset creation we put a threshold 2k for context length (input token limit) but we dont have enough VRAM unfortunately it will take a lot of VRAM to put everything into memory so we are just gonna stop at 512
    tokenizer=tokenizer,
    packing = False,
    args=training_arguments,
)

Map (num_proc=2): 100%|██████████| 81491/81491 [00:21<00:00, 3852.41 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [45]:
tokenizer.pad_token=tokenizer.eos_token
# tokenizer.padding_side="left"

In [46]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/envs/pii_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
1,2.036200
2,1.761900
3,1.850600
4,2.152700
5,1.357400
6,1.209700
7,1.220000
8,1.067800
9,1.233800
10,1.094400


/opt/conda/envs/pii_env/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [47]:
# Empty VRAM
del model
# del pipe
del trainer
import gc
gc.collect()

48284

In [48]:
###Merge the Base Model with the Trained Adapter
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

#Reload the Base Model and load the QLoRA adapters
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [49]:
model.save_pretrained(final_model_name)
tokenizer.save_pretrained(final_model_name)

('pii-llama3-8b-4bit-merged/tokenizer_config.json',
 'pii-llama3-8b-4bit-merged/special_tokens_map.json',
 'pii-llama3-8b-4bit-merged/tokenizer.json')

In [50]:
del model

## vLLM Test Model

### Load Model

In [29]:
%pip install spacy
%pip install kaleido python-multipart typing-extensions
%pip install vllm
%pip install huggingface_hub
%pip install flash-attn
%pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [19]:
from transformers import AutoTokenizer
from spacy.lang.en import English
from huggingface_hub import login
from vllm import LLM, SamplingParams
import transformers
import torch
import os
import json
import re

2024-05-10 06:12:56,047	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [20]:
llm = LLM(model=final_model_name, gpu_memory_utilization=0.95, max_context_len_to_capture=2048)

INFO 05-10 06:12:56 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='pii-llama3-8b-4bit-merged', tokenizer='pii-llama3-8b-4bit-merged', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 05-10 06:12:56 utils.py:357] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 05-10 06:12:56 selector.py:16] Using FlashAttention backend.
INFO 05-10 06:12:59 model_runner.py:104] Loading model weights took 14.9595 GB
INFO 05-10 06:13:00 gpu_executor.py:94] # GPU blocks: 2811, # CPU blocks: 2048
INFO 05-10 06:13:00 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-10 06:13:00 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-10 06:13:11 model_runner.py:867] Graph capturing finished in 11 secs.


### Prepare Dataset

In [86]:
max_test_samples = 30 # Set to len(pii_test_dataset) for testng on full dataset

selected_test_indexes = set(random.sample(range(len(pii_test_dataset)), max_test_samples))
capped_pii_test_dataset = Dataset.from_pandas(pii_test_dataset)
capped_pii_test_dataset = capped_pii_test_dataset.select([i for i in range(len(capped_pii_test_dataset)) if i in selected_test_indexes])
capped_pii_test_dataset

Dataset({
    features: ['text', 'output', 'full_text', 'source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'final_text', 'testing_text', '__index_level_0__'],
    num_rows: 30
})

In [87]:
# Counting number of occurances of each label
pii_labels = ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS']
num_labels = {label: 0 for label in pii_labels}

for label in pii_labels:
  for output in capped_pii_test_dataset['output']:
    num_labels[label] += output.count(label)

print("Capped Test Dataset PII Occurrances:\n")
num_labels

Capped Test Dataset PII Occurrances:



{'NAME_STUDENT': 5,
 'EMAIL': 6,
 'USERNAME': 14,
 'ID_NUM': 30,
 'PHONE_NUM': 9,
 'URL_PERSONAL': 2,
 'STREET_ADDRESS': 23}

### Test One Sample

In [88]:
# Test Input
input_text = "Juan Rosa recently moved to 22233 Stante Run and can be reached at 290-828-2909 for inquiries about the community book club."
print(input_text)

Juan Rosa recently moved to 22233 Stante Run and can be reached at 290-828-2909 for inquiries about the community book club.


In [89]:
# Setting Hyperparameters
sampling_params = SamplingParams(temperature=0, max_tokens=512)

# Testing Model
outputs = llm.generate(
    [format_prompt(input_text)],
    sampling_params
    )

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print()
    print("Generated text:\n", generated_text)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


Generated text:
 List of Names:
* Juan Rosa

List of Emails:
* None

List of Usernames:
* None

List of ID Numbers:
* None

List of Phone Numbers:
* 290-828-2909

List of URLs:
* None

List of Street Addresses:
* 22233 Stante Run



##

### Test All Evaluation Samples

In [90]:
pii_categories = ['Names', 'Emails', 'Usernames', 'ID Numbers', 'Phone Numbers', 'URLs', 'Street Addresses']
pii_categories_pattern = '|'.join(pii_categories)
category_to_label = {'Names': 'NAME_STUDENT', 'Emails': 'EMAIL', 'Usernames': 'USERNAME', 'ID Numbers': 'ID_NUM', 'Phone Numbers': 'PHONE_NUM', 'URLs': 'URL_PERSONAL', 'Street Addresses': 'STREET_ADDRESS'}


In [91]:
# Convert new output format to previous output
def convert_output_format(text):
    outputs = []
    pii_outputs = re.findall(f"({pii_categories_pattern}):\n((\*.+?\n)+)", text, re.DOTALL)
    for category, data, _ in pii_outputs:
        data = data.replace("* ", "").strip().split("\n")

        for item in data:
            if item == 'None': continue
            outputs.append(f"{item} ({category_to_label[category]})")

    return outputs

In [92]:
nlp = English()
english_tokenizer = nlp.tokenizer

In [93]:
# Predict

model_outputs = llm.generate(
    [format_prompt(input) for input in capped_pii_test_dataset['full_text']],
    sampling_params
)

Processed prompts: 100%|██████████| 30/30 [00:31<00:00,  1.04s/it]


In [94]:
total_classifications = 0
num_correct_classifications = 0
num_hallucinated = 0
# Evaluate
for id, (input_text, expected_output, model_output, expected_text) in enumerate(zip(capped_pii_test_dataset['full_text'], capped_pii_test_dataset['output'], model_outputs, capped_pii_test_dataset['text'])):
    expected_output_list = re.split(r',?\n', expected_output)
    expected_output_list = [re.sub(r'((^\d+\.)|(\*))\s*', '', output) for output in expected_output_list] # Remove "* " or "1. " if values starts with them
    expected_output_list_lowercased = [re.sub(r'((^\d+\.)|(\*))\s*', '', output).lower().strip() for output in expected_output_list] # Remove "* " or "1. " if values starts with them
    generated_text = model_output.outputs[0].text

    # New Code Added:
    if len(expected_output_list) == 1 and expected_output_list[0] == '':
        expected_output_list = []
        expected_output_list_lowercased = []

    # Process output text
    outputs: list[str] = convert_output_format(generated_text)
    print("Expected Outputs:\n", expected_output_list)
    print("Outputs:\n", outputs)

    # Total Classifications should be equal to the list of expected outputs
    total_classifications += len(expected_output_list)

    for output in outputs:
        if output.lower().strip() in expected_output_list_lowercased:
            num_correct_classifications += 1
        else:
            print("Hallucinated:", output)
            num_hallucinated += 1
    print()

# TODO: Calculate test accuracy with Rogue-L metric: rogue(pii_dataset_test['text'][i].replace(format_prompt(input)), model_output.outputs[i].text) 

# If all test samples do not have any pii
if(total_classifications == 0 and num_correct_classifications == 0):
    total_classifications = 1
    num_correct_classifications = 1

print("Accuracy:", round(num_correct_classifications / total_classifications, 3))
print("Number Hallucinated:", num_hallucinated)

Expected Outputs:
 []
Outputs:
 []

Expected Outputs:
 []
Outputs:
 []

Expected Outputs:
 []
Outputs:
 []

Expected Outputs:
 ['1944A (USERNAME)', '1940andreea-emilia.bertusi (USERNAME)', '']
Outputs:
 ['1944A@outlook.com (EMAIL)', 'andreea-emilia.bertusi@protonmail.com (EMAIL)', 'KP85998GP (ID_NUM)', 'BD02101CX (ID_NUM)', '587-102-8153 (PHONE_NUM)', '+47-265 700 1910 (PHONE_NUM)', '0104 585 327-3633 (STREET_ADDRESS)', '%H58mNaa\\\\f+I (STREET_ADDRESS)', '18:39 (STREET_ADDRESS)', '17:18:49 (STREET_ADDRESS)', '13/04/2038 (STREET_ADDRESS)', '20/12/1940 (STREET_ADDRESS)', '4\\1944A\\1944A (STREET_ADDRESS)', '1940andreea-emilia.bertusi (STREET_ADDRESS)', '1940andreea-emilia.bertusi@protonmail.com (STREET_ADDRESS)', '1940andreea-emilia.bertusi@protonmail.com (STREET_ADDRESS)', '1940andreea-emilia.bertusi@protonmail.com (STREET_ADDRESS)', '1940andreea-emilia.bertusi@protonmail.com (STREET_ADDRESS)', '1940andreea-emilia.bertusi@protonmail.com (STREET_ADDRESS)', '1940andreea-emilia.bertusi@pr

In [30]:
total_classifications

0